In [ ]:
# import gradio as gr
import pandas as pd
from datetime import datetime
import joblib
import streamlit as st

import numpy as np

# Para conexión a base de datos PostgreSQL via ODBC
import pyodbc

import wandb
from ICBAnalytics import GetData

# Para webservice
from io import StringIO
import requests

import warnings

warnings.filterwarnings("ignore")


In [ ]:
# import gradio as gr
import pandas as pd
from datetime import datetime
import joblib
import streamlit as st

import numpy as np

# Para conexión a base de datos PostgreSQL via ODBC
import pyodbc

import wandb
from ICBAnalytics import GetData

# Para webservice
from io import StringIO
import requests

import warnings

warnings.filterwarnings("ignore")


def modelo():
    weekday = datetime.today().strftime("%Y-%m-%d")

    def test_pred():
        # Nos conectamos a Webservice para solicitar las rutas y gestores activos para la fecha de análisis.
        first_day = datetime.today()
        first_day = datetime(first_day.year, first_day.month, 1).date()
        today = datetime.today().date()

        dates = pd.date_range(first_day, today)

        payload = {}
        headers = {
            "X-ICB-TOKEN": "7ad88085-82d5-4524-927f-406820f4ec51",
            "Cookie": "sap-usercontext=sap-client=300",
        }

        rutas_bd = pd.DataFrame()

        for date in dates:
            date_str = date.strftime("%Y%m%d")
            url = f"https://papas.icb.cl:44330/zapi/mdt/get_n?select=VKGRP,STCD1&from=RutaClienteVdd&fmt=txt&where=DATBI>={date_str},DATAB<={date_str},VKORG=1000"
            response = requests.request("GET", url, headers=headers, data=payload)
            # Convert String into StringIO
            csvStringIO = StringIO(response.text)
            rutas_bd = pd.concat(
                [rutas_bd, pd.read_csv(csvStringIO, sep="\t", header=0)]
            )

        rutas_bd.reset_index(drop=True, inplace=True)
        rutas_bd.drop_duplicates(subset=["VKGRP"], keep="last", inplace=True)
        rutas_bd.reset_index(drop=True, inplace=True)

        rutas_bd.rename(
            columns={"VKGRP": "Ruta", "STCD1": "Rut Gestor Responsable"}, inplace=True
        )
        rutas_bd["Rut Gestor Responsable"] = rutas_bd["Rut Gestor Responsable"].str.upper()
        rutas_bd["Ruta"] = rutas_bd["Ruta"].astype(str)

        rutas_3_o_mas_inv = (
            GetData()
            .get_data_postgresql(
            """
            select "Ruta" from inventarios i 
            where 
                extract(month from current_date) = extract(month from fecha)
                and extract(year from current_date) = extract(year from fecha)
            group by "Ruta"
            having count(*) >= 3;
            """
            )["Ruta"]
            .to_list()
        )

        print("Rutas con 3 o más inventarios:", len(rutas_3_o_mas_inv))
        rutas_bd = rutas_bd[~rutas_bd["Ruta"].isin(rutas_3_o_mas_inv)]

        # Rutas de ventas al personal
        rutas_vp = ["832", "973", "991", "B18", "B38", "B98", "C19"]

        rutas_vp_inv = (
            GetData()
            .get_data_postgresql(
                f"""
            select "Ruta" from inventarios i 
            where 
                extract(month from current_date) = extract(month from fecha)
                and extract(year from current_date) = extract(year from fecha)
                and "Ruta" IN ({str(rutas_vp).strip("[]")})
            group by "Ruta"
            having count(*) >= 1;
            """
            )["Ruta"]
            .to_list()
        )

        rutas_bd = rutas_bd[~rutas_bd["Ruta"].isin(rutas_vp_inv)]

        #     data = pd.read_csv(csv_file)
        rutas = rutas_bd["Ruta"].unique().tolist()
        print("Rutas:", len(rutas))

        query = f"""
        select "Grupo de vendedores", 
        EXTRACT('year' FROM "Fecha de factura para el índice de factura e impresión")::NUMERIC "Año", 
        EXTRACT('month' FROM "Fecha de factura para el índice de factura e impresión") "Mes", 
        AVG("Valor neto de posición de factura en moneda de documento" / "Cantidad facturada verdaderamente") "Precio promedio",
        AVG(
            CASE
                WHEN ("Unidad de peso" = 'G') THEN "Peso neto" / 1000
                WHEN ("Unidad de peso" = 'KG') THEN "Peso neto" * 1
                ELSE 0
            END
        ) as peso_promedio
        from VENTA_CT VC 
        where "Grupo de vendedores" IN ({str(rutas).strip("[]")})
        group by 
            "Grupo de vendedores", 
            EXTRACT('year' FROM "Fecha de factura para el índice de factura e impresión")::NUMERIC, 
            EXTRACT('month' FROM "Fecha de factura para el índice de factura e impresión")
        """
        venta = GetData().get_data_postgresql(query)

        query = """
        SELECT 
            "Ruta",
            "MES",
            "Ruta con diferencia",
            "Ruta con diferencia +1",
            "fecha",
            "Valor total anterior",
            "Tipo Inventario",
            "inventariado",
            "Vendedor"
        FROM inv_ampliado
        WHERE inv_ampliado."Ruta" = ANY(ARRAY{})
        ORDER BY "fecha" DESC
        """.format(
            rutas
        )
        data = GetData().get_data_postgresql(query)
        print("{} Total records in the query!".format(data.shape[0]))

        data["Inventario del mes"] = data.groupby(["Ruta", "MES"]).cumcount() + 1
        data["Inventario del mes"] = data.groupby(["Ruta", "MES"]).cumcount() + 1
        #data.dropna(subset=["Ruta con diferencia +1"], inplace=True)
        data["fecha"] = pd.to_datetime(data["fecha"])
        data["Mes"] = data["fecha"].apply(lambda x: x.month)
        data["Año"] = data["fecha"].apply(lambda x: x.year)
        data["Inventario del mes"] = np.where(
            data["Inventario del mes"] <= 3,
            data["Inventario del mes"].apply(lambda x: str(x)),
            "4+",
        )

        data = data.merge(rutas_bd, left_on="Ruta", right_on="Ruta", how="outer")
        data["rut_gestor"] = data["Rut Gestor Responsable"]

        consolidado_mensual = data.merge(
            venta,
            left_on  = ["Ruta", "Año", "Mes"],
            right_on = ["Grupo de vendedores", "Año", "Mes"],
            how      = "left",
        )
        print("Rutas consolidado:", consolidado_mensual["Ruta"].nunique())
        consolidado_mensual["inventariado"] = np.where(
            consolidado_mensual["inventariado"] == "Santiago enea",
            "Santiago Enea",
            consolidado_mensual["inventariado"],
        )

        cat_cols = [
            "Ruta con diferencia",
            "inventariado",
            "Tipo Inventario",
            "Inventario del mes",
        ]

        for c in cat_cols:
            consolidado_mensual[c] = consolidado_mensual[c].apply(lambda x: str(x))

        # Verificamos la fecha de hoy para tomar solo los resultados de este mes.
        # today = date.today().strftime('%Y-%m-%d')
        # weekday = date.strftime('%Y-%m-%d')
        inventarios_mes = data[
            data["fecha"].apply(
                lambda x: (
                    True
                    if x.month == pd.to_datetime(weekday, format="%Y-%m-%d").month
                    else False
                )
                & (
                    True
                    if x.year == pd.to_datetime(weekday, format="%Y-%m-%d").year
                    else False
                )
            )
        ]
        inventarios_totales = data.copy()

        indexes = consolidado_mensual.reset_index().groupby(["Ruta"])["index"].first()
        print(consolidado_mensual.columns)
        new_df = consolidado_mensual.iloc[indexes].rename(
            columns={
                "Ruta con diferencia": "ruta_con_diferencia",
                "Tipo Inventario": "tipo_inventario",
                "Inventario del mes": "inventario_del_mes",
                "Valor total anterior": "valor_total_anterior",
                "Precio promedio": "precio_promedio",
            }
        )

        api = wandb.Api()
        artifact = api.artifact(
            "icb-analytics/rutas-con-diferencias-de-inventario/RandomForest:v6"
        )
        model_path = artifact.download() + "\\model.pkl"
        model = joblib.load(model_path)

        predictors = [
            "valor_total_anterior",
            "tipo_inventario",
            "inventariado",
            "inventario_del_mes",
            "precio_promedio",
            "ruta_con_diferencia",
        ]
        df_output = new_df[
            ["Ruta", "Vendedor", "Rut Gestor Responsable", "inventariado"]
        ]
        df_output["Ruta +D"] = model.predict(new_df[predictors])
        df_output["Ruta +D Prob."] = model.predict_proba(new_df[predictors])[:, 1]
        df_output.rename(columns={"inventariado": "Zona inventario"}, inplace=True)
        # df_output = df_output[df_output['Ruta +D'] == 1]
        df_output["Ranking"] = (
            df_output["Ruta +D Prob."]
            .rank(method="first", ascending=False)
            .astype("int")
        )
        print("Rutas con predicción:", df_output["Ruta"].nunique())
        df_output.reset_index(inplace=True, drop=True)
        df_output.sort_values("Zona inventario", ascending=True, inplace=True)

        return df_output, inventarios_mes, inventarios_totales, new_df

    df_output, inventarios_mes, inventarios_totales, new_df = test_pred()
    ## Verificar si ya se le ha realizado inventario en el mes

    inventarios_mes_completo = inventarios_mes[
        inventarios_mes["Tipo Inventario"] == "1"
    ]
    inventarios_mes_completo = (
        inventarios_mes_completo.drop_duplicates(subset=["Ruta", "inventariado"])
        .groupby(["inventariado"])["Ruta"]
        .unique()
        .reset_index()
    )
    inventarios_mes_completo = inventarios_mes_completo.rename(
        columns={"Ruta": "rutas_inventariadas"}
    )

    df_inv = df_output.merge(
        right    = inventarios_mes_completo,
        how      = "left",
        left_on  = "Zona inventario",
        right_on = "inventariado",
    )

    df_inv["rutas_inventariadas"] = np.where(
        df_inv["rutas_inventariadas"].isna(),
        pd.Series([[] for x in range(df_inv.shape[0])]),
        df_inv["rutas_inventariadas"],
    )

    df_inv["Prioridad"] = np.select(
        condlist=[
            (df_inv["Ruta +D"] == 1)
            & (
                df_inv.apply(
                    lambda x: not (x["Ruta"] in x["rutas_inventariadas"]), axis=1
                )
            ),
            (df_inv["Ruta +D"] == 0)
            & (
                df_inv.apply(
                    lambda x: not (x["Ruta"] in x["rutas_inventariadas"]), axis=1
                )
            ),
            (df_inv["Ruta +D"] == 1)
            & (df_inv.apply(lambda x: x["Ruta"] in x["rutas_inventariadas"], axis=1)),
        ],
        choicelist=[1, 2, 3],
        default=4,
    )
    df_inv = df_inv.sort_values(["Zona inventario", "Prioridad", "Ranking"])#.drop(
    #    columns=["rutas_inventariadas", "Ruta +D Prob.", "inventariado"]
    #)
    return inventarios_totales, inventarios_mes, df_inv
    

    # Verificar la capacidad

    query = """
    select "Zona inventariado", "Prom inv semanal" from CAPACIDAD_ZONAS_OBJETIVO CZO;
    """
    capacidad_df = GetData().get_data_postgresql(query)
    print("{} Total records in the query!".format(capacidad_df.shape[0]))

    prior = 4

    capacidad_df["Prom inv semanal"] = capacidad_df["Prom inv semanal"].astype(int)
    capacidades = capacidad_df.set_index("Zona inventariado").to_dict()[
        "Prom inv semanal"
    ]

    df_inv_final = pd.DataFrame()

    for zona, capacidad in capacidades.items():
        prior = 1
        cap = capacidad
        while (cap > 0) and (prior <= 4):
            df_prior = df_inv[
                (df_inv["Zona inventario"] == zona) & (df_inv["Prioridad"] == prior)
            ]
            if cap > df_prior.shape[0]:
                df_inv_final = pd.concat([df_inv_final, df_prior])
                cap -= df_prior.shape[0]
            else:
                df_inv_final = pd.concat([df_inv_final, df_prior.iloc[:cap,]])
                cap = 0
            prior += 1

    # Ajustar para Postgres

    semana = pd.to_datetime(weekday).week
    año = pd.to_datetime(weekday).year
    # semana = pd.to_datetime(today).week
    # año = pd.to_datetime(today).year
    df_pred = df_inv_final.copy()

    df_pred["Fecha"] = weekday
    df_pred["Semana"] = semana
    df_pred["Año"] = año

    print("df_pred columns:", df_pred.columns.to_list())
    df_pred = df_pred[
        [
            "Ruta",
            "Vendedor",
            "Zona inventario",
            "Ruta +D",
            "Fecha",
            "Semana",
            "Año",
            "Prioridad",
            "Rut Gestor Responsable",
        ]
    ]

    lista_zonas = df_pred[df_pred["Prioridad"] == 2]["Zona inventario"].unique()
    index_actualizar = df_pred[
        (df_pred["Prioridad"] == 3)
        & (df_pred["Zona inventario"].apply(lambda x: x not in lista_zonas))
    ].index.to_list()
    df_pred.loc[index_actualizar, "Prioridad"] = 1

    def carga_pred(df_pred):
        try:
            server = "localhost,5432"
            database = "comercial"  # enter database name
            user = "postgres"
            pswd = "M@rSopa.1!"
            cnxn = pyodbc.connect(
                "DRIVER={PostgreSQL ODBC Driver(UNICODE)};SERVER="
                + server
                + ";DATABASE="
                + database
                + ";Trusted_Connection=yes;UID="
                + user
                + "; PWD="
                + pswd
            )
            cursor = cnxn.cursor()

        except Exception as e:
            print(
                "Failed to connect to the database {%} in the server {%}}".format(
                    database, server
                ),
                e,
            )

        try:
            query = """
                INSERT INTO public.pred_inventario
                ("Ruta", "Vendedor", "Zona inventario", "Ruta +D", "Fecha", "Semana", "Año", prioridad, rut_gestor)
                VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?);
            """
            cursor.fast_executemany = True
            cursor.executemany(query, df_pred.values.tolist())
            cursor.commit()
            print(
                "Query updated succesfully! {} Rows inserted".format(
                    len(df_pred.values)
                )
            )
        except Exception as e:
            print("Failed to call the query.", e)
        finally:
            # closing database connection.
            if cnxn:
                cursor.close()
                cnxn.close()
                print("PostgreSQL connection is closed")

    carga_pred(df_pred)

In [5]:
first_day = datetime.today()
first_day = datetime(first_day.year, first_day.month, 1).date()
end = datetime.today()
today = datetime.today().date()

dates = pd.date_range(first_day, end)
print(dates)

payload = {}
headers = {
    "X-ICB-TOKEN": "7ad88085-82d5-4524-927f-406820f4ec51",
    "Cookie": "sap-usercontext=sap-client=300",
}

rutas_bd = pd.DataFrame()

for date in dates:
    date_str = date.strftime("%Y%m%d")
    url = f"https://papas.icb.cl:44330/zapi/mdt/get_n?select=VKGRP,STCD1&from=RutaClienteVdd&fmt=txt&where=DATBI>={date_str},DATAB<={date_str},VKORG=1000"
    response = requests.request("GET", url, headers=headers, data=payload)
    # Convert String into StringIO
    csvStringIO = StringIO(response.text)
    rutas_bd = pd.concat(
        [rutas_bd, pd.read_csv(csvStringIO, sep="\t", header=0)]
    )

DatetimeIndex(['2024-12-01', '2024-12-02', '2024-12-03', '2024-12-04',
               '2024-12-05', '2024-12-06', '2024-12-07', '2024-12-08',
               '2024-12-09'],
              dtype='datetime64[ns]', freq='D')


In [6]:
rutas_bd.drop_duplicates(["VKGRP"], keep="last", inplace=True)

In [ ]:
rutas_nov = [
    "630",
    "698",
    "866",
    "887",
    "976",
    "A05",
    "A48",
    "A52",
    "A60",
    "A89",
    "B12",
    "B46",
    "B74",
    "B78",
    "B82",
    "B89",
    "C21",
    "C33",
    "C34",
    "C36",
    "C37",
    "C48",
    "C56",
    "C76",
    "C83"
]

In [8]:
rutas = rutas_bd[
    rutas_bd["VKGRP"].isin(rutas_nov)
]["VKGRP"].sort_values().to_list()

In [9]:
rutas

['630',
 '698',
 '887',
 '976',
 'A05',
 'A60',
 'A89',
 'B46',
 'B74',
 'B89',
 'C33',
 'C37',
 'C48',
 'C56',
 'C76',
 'C83']

In [10]:
import pandas as pd
from io import StringIO
import requests

payload = {}
headers = {
    "X-ICB-TOKEN": "7ad88085-82d5-4524-927f-406820f4ec51",
    "Cookie": "sap-usercontext=sap-client=300",
}

url = f"https://papas.icb.cl:44330/zapi/mdt/get_n?select=all&from=RutaClienteVdd&fmt=json,VKORG=1000"
response = requests.request("GET", url, headers=headers, data=payload)
# Convert String into StringIO
rutas = pd.DataFrame(response.json())

In [12]:
rutas

,IdOrgVta,IdVendedor,valido_de,fin_validez,Rut,vendedor_ruta,comentario,telefono_movil,region,poblacion,distrito,Direccion,Numero,Mail,responsable_modificacion,fecha_modificacion,hora_modificacion
0,1000,600,2016-09-14,2016-11-21,15596288-7,VR0092,Vendedor Renuncia 21-11,,,,,,,,,0000-00-00,00:00:00
1,1000,600,2016-11-22,2016-12-31,18970966-8,VR0113,renuncia,,13,SANTIAGO,SANTIAGO,Caupolican,9401,nulo@icb.cl,,0000-00-00,00:00:00
2,1000,600,2017-01-01,2018-03-05,17625527-7,VR0131,renuncia 05.Marzo,,13,SANTIAGO,SANTIAGO,Caupolican,9401,nulo@icb.cl,,0000-00-00,00:00:00
3,1000,600,2018-03-09,2018-06-07,17451906-4,VR0321,desvinculado,,13,SANTIAGO,SANTIAGO,Caupolican,9401,nulo@icb.cl,,0000-00-00,00:00:00
4,1000,600,2018-06-08,2019-01-02,18353393-2,VR0044,desvinculado,,13,SANTIAGO,SANTIAGO,Caupolican,9401,nulo@icb.cl,,0000-00-00,00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7915,1000,630,2024-11-01,2024-12-02,18485481-3,VR0977,Ruta eliminada,,13,QUILICURA,QUILICURA,CAUPOLICAN 9401,,VENDEDOR@ICB.CL,DCEPEDA,2024-12-12,17:38:05
7916,1000,A17,2023-09-26,2024-11-29,18197544-K,VR1221,Desvinculado,,13,QUILICURA,QUILICURA,CAUPOLICAN,9401,VENDEDOR@ICB.CL,DCEPEDA,2024-12-13,10:02:57
7917,1000,A17,2024-12-02,9999-12-31,18195461-2,VR1585,Asume ruta,,13,QUILICURA,QUILICURA,CAUPOLICÁN,9401,NOTIENE@ICB.CL,DCEPEDA,2024-12-13,10:04:02
7918,1000,939,2024-09-20,2024-12-15,11118734-7,VR1126,Vacaciones titular,,13,QUILICURA,QUILICURA,CAUPOLICAN 9401,,VENDEDOR@ICB.CL,DCEPEDA,2024-12-13,12:09:51


In [27]:
from google.cloud import bigquery
client = bigquery.Client() 
inv = client.query(f"""
    SELECT min(Fecha) FROM `RutasCD.pred_inventario` WHERE Fecha > '2024-11-01'        
""").result().to_dataframe()